In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import LSTM
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[3], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [9]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [11]:
run_experiment(X,y)

Epoch 1/50
8/8 [==============================] - 0s 51ms/step - loss: 1.3926 - accuracy: 0.4863
Epoch 2/50
8/8 [==============================] - 0s 50ms/step - loss: 1.2986 - accuracy: 0.5411
Epoch 3/50
8/8 [==============================] - 0s 49ms/step - loss: 1.2566 - accuracy: 0.5181
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 1.1971 - accuracy: 0.5433
Epoch 5/50
8/8 [==============================] - 0s 49ms/step - loss: 1.1562 - accuracy: 0.5225
Epoch 6/50
8/8 [==============================] - 0s 49ms/step - loss: 1.1165 - accuracy: 0.5400
Epoch 7/50
8/8 [==============================] - 0s 49ms/step - loss: 1.0775 - accuracy: 0.5312
Epoch 8/50
8/8 [==============================] - 0s 49ms/step - loss: 1.0525 - accuracy: 0.5159
Epoch 9/50
8/8 [==============================] - 0s 49ms/step - loss: 1.0213 - accuracy: 0.4929
Epoch 10/50
8/8 [==============================] - 0s 49ms/step - loss: 0.9947 - accuracy: 0.4907
Epoch 11/50
8/8 [============

8/8 [==============================] - 0s 48ms/step - loss: 0.6556 - accuracy: 0.6692
Epoch 35/50
8/8 [==============================] - 0s 48ms/step - loss: 0.6007 - accuracy: 0.7360
Epoch 36/50
8/8 [==============================] - 0s 50ms/step - loss: 0.5726 - accuracy: 0.7568
Epoch 37/50
8/8 [==============================] - 0s 49ms/step - loss: 0.5013 - accuracy: 0.8160
Epoch 38/50
8/8 [==============================] - 0s 50ms/step - loss: 0.4774 - accuracy: 0.8412
Epoch 39/50
8/8 [==============================] - 0s 50ms/step - loss: 0.4497 - accuracy: 0.8291
Epoch 40/50
8/8 [==============================] - 0s 50ms/step - loss: 0.4119 - accuracy: 0.8554
Epoch 41/50
8/8 [==============================] - 0s 50ms/step - loss: 0.3743 - accuracy: 0.8653
Epoch 42/50
8/8 [==============================] - 0s 49ms/step - loss: 0.4588 - accuracy: 0.8127
Epoch 43/50
8/8 [==============================] - 0s 50ms/step - loss: 0.3740 - accuracy: 0.8642
Epoch 44/50
8/8 [===============

8/8 [==============================] - 0s 48ms/step - loss: 0.8199 - accuracy: 0.6353
Epoch 18/50
8/8 [==============================] - 0s 48ms/step - loss: 0.7661 - accuracy: 0.7076
Epoch 19/50
8/8 [==============================] - 0s 48ms/step - loss: 0.7322 - accuracy: 0.7054
Epoch 20/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7266 - accuracy: 0.7010
Epoch 21/50
8/8 [==============================] - 0s 48ms/step - loss: 0.6487 - accuracy: 0.7623
Epoch 22/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6027 - accuracy: 0.7820
Epoch 23/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6779 - accuracy: 0.7404
Epoch 24/50
8/8 [==============================] - 0s 50ms/step - loss: 0.5571 - accuracy: 0.7886
Epoch 25/50
8/8 [==============================] - 0s 50ms/step - loss: 0.5733 - accuracy: 0.7755
Epoch 26/50
8/8 [==============================] - 0s 50ms/step - loss: 0.5379 - accuracy: 0.8193
Epoch 27/50
8/8 [===============

8/8 [==============================] - 0s 49ms/step - loss: 0.7084 - accuracy: 0.5257
 >#5: 61.572 
[61.57205104827881, 94.3231463432312, 61.57205104827881, 96.50654792785645, 61.57205104827881]
 Accuracy: 75.109% (+/-16.594) 


In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3),input_shape=(None,n_length,n_features)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(50,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    
run_experiment_2(X,y)

Epoch 1/50
8/8 [==============================] - 0s 52ms/step - loss: 1.4192 - accuracy: 0.4819
Epoch 2/50
8/8 [==============================] - 0s 50ms/step - loss: 1.3336 - accuracy: 0.5148
Epoch 3/50
8/8 [==============================] - 0s 50ms/step - loss: 1.2708 - accuracy: 0.5268
Epoch 4/50
8/8 [==============================] - 0s 49ms/step - loss: 1.2218 - accuracy: 0.5356
Epoch 5/50
8/8 [==============================] - 0s 49ms/step - loss: 1.1951 - accuracy: 0.5148
Epoch 6/50
8/8 [==============================] - 0s 50ms/step - loss: 1.1523 - accuracy: 0.5225
Epoch 7/50
8/8 [==============================] - 0s 48ms/step - loss: 1.1259 - accuracy: 0.4808
Epoch 8/50
8/8 [==============================] - 0s 51ms/step - loss: 1.0875 - accuracy: 0.5071
Epoch 9/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0622 - accuracy: 0.4973
Epoch 10/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0262 - accuracy: 0.5279
Epoch 11/50
8/8 [============

Epoch 27/50
8/8 [==============================] - 0s 49ms/step - loss: 0.3888 - accuracy: 0.8817
Epoch 28/50
8/8 [==============================] - 0s 51ms/step - loss: 0.3503 - accuracy: 0.8970
Epoch 29/50
8/8 [==============================] - 0s 51ms/step - loss: 0.3378 - accuracy: 0.8981
Epoch 30/50
8/8 [==============================] - 0s 50ms/step - loss: 0.3869 - accuracy: 0.8828
Epoch 31/50
8/8 [==============================] - 0s 51ms/step - loss: 0.3793 - accuracy: 0.8751
Epoch 32/50
8/8 [==============================] - 0s 48ms/step - loss: 0.3258 - accuracy: 0.9179
Epoch 33/50
8/8 [==============================] - 0s 50ms/step - loss: 0.3276 - accuracy: 0.8938
Epoch 34/50
8/8 [==============================] - 0s 51ms/step - loss: 0.4042 - accuracy: 0.8423
Epoch 35/50
8/8 [==============================] - 0s 50ms/step - loss: 0.3796 - accuracy: 0.8653
Epoch 36/50
8/8 [==============================] - 0s 51ms/step - loss: 0.3533 - accuracy: 0.8697
Epoch 37/50
8/8 [===

 >#3: 72.489 
Epoch 1/50
8/8 [==============================] - 0s 50ms/step - loss: 1.3507 - accuracy: 0.5378
Epoch 2/50
8/8 [==============================] - 0s 50ms/step - loss: 1.3228 - accuracy: 0.4973
Epoch 3/50
8/8 [==============================] - 0s 48ms/step - loss: 1.2647 - accuracy: 0.5279
Epoch 4/50
8/8 [==============================] - 0s 49ms/step - loss: 1.2180 - accuracy: 0.5093
Epoch 5/50
8/8 [==============================] - 0s 50ms/step - loss: 1.1667 - accuracy: 0.5389
Epoch 6/50
8/8 [==============================] - 0s 50ms/step - loss: 1.1311 - accuracy: 0.5334
Epoch 7/50
8/8 [==============================] - 0s 51ms/step - loss: 1.0920 - accuracy: 0.5246
Epoch 8/50
8/8 [==============================] - 0s 50ms/step - loss: 1.0748 - accuracy: 0.5104
Epoch 9/50
8/8 [==============================] - 0s 51ms/step - loss: 1.0337 - accuracy: 0.5586
Epoch 10/50
8/8 [==============================] - 0s 51ms/step - loss: 0.9835 - accuracy: 0.5816
Epoch 11/50
8/8

8/8 [==============================] - 0s 50ms/step - loss: 0.2534 - accuracy: 0.9452
Epoch 27/50
8/8 [==============================] - 0s 51ms/step - loss: 0.2358 - accuracy: 0.9485
Epoch 28/50
8/8 [==============================] - 0s 51ms/step - loss: 0.2048 - accuracy: 0.9617
Epoch 29/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1952 - accuracy: 0.9628
Epoch 30/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1861 - accuracy: 0.9671
Epoch 31/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1679 - accuracy: 0.9715
Epoch 32/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1596 - accuracy: 0.9671
Epoch 33/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1391 - accuracy: 0.9759
Epoch 34/50
8/8 [==============================] - 0s 49ms/step - loss: 0.1191 - accuracy: 0.9836
Epoch 35/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1270 - accuracy: 0.9792
Epoch 36/50
8/8 [===============

In [13]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='selu'))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation= 'selu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_3(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    
run_experiment_3(X,y)

Epoch 1/50
8/8 [==============================] - 0s 51ms/step - loss: 1.5659 - accuracy: 0.5192
Epoch 2/50
8/8 [==============================] - 0s 50ms/step - loss: 1.4882 - accuracy: 0.4896
Epoch 3/50
8/8 [==============================] - 0s 50ms/step - loss: 1.4454 - accuracy: 0.4962
Epoch 4/50
8/8 [==============================] - 0s 49ms/step - loss: 1.4014 - accuracy: 0.5060
Epoch 5/50
8/8 [==============================] - 0s 51ms/step - loss: 1.3409 - accuracy: 0.5148
Epoch 6/50
8/8 [==============================] - 0s 50ms/step - loss: 1.3058 - accuracy: 0.5400
Epoch 7/50
8/8 [==============================] - 0s 49ms/step - loss: 1.2957 - accuracy: 0.5268
Epoch 8/50
8/8 [==============================] - 0s 49ms/step - loss: 1.2833 - accuracy: 0.5290
Epoch 9/50
8/8 [==============================] - 0s 50ms/step - loss: 1.2741 - accuracy: 0.5279
Epoch 10/50
8/8 [==============================] - 0s 50ms/step - loss: 1.2722 - accuracy: 0.5038
Epoch 11/50
8/8 [============

Epoch 27/50
8/8 [==============================] - 0s 52ms/step - loss: 0.5682 - accuracy: 0.8784
Epoch 28/50
8/8 [==============================] - 0s 50ms/step - loss: 0.5758 - accuracy: 0.8817
Epoch 29/50
8/8 [==============================] - 0s 51ms/step - loss: 0.5328 - accuracy: 0.8894
Epoch 30/50
8/8 [==============================] - 0s 50ms/step - loss: 0.5493 - accuracy: 0.8916
Epoch 31/50
8/8 [==============================] - 0s 50ms/step - loss: 0.4780 - accuracy: 0.9124
Epoch 32/50
8/8 [==============================] - 0s 50ms/step - loss: 0.4802 - accuracy: 0.8981
Epoch 33/50
8/8 [==============================] - 0s 49ms/step - loss: 0.4401 - accuracy: 0.9168
Epoch 34/50
8/8 [==============================] - 0s 50ms/step - loss: 0.4585 - accuracy: 0.9146
Epoch 35/50
8/8 [==============================] - 0s 51ms/step - loss: 0.4437 - accuracy: 0.9124
Epoch 36/50
8/8 [==============================] - 0s 50ms/step - loss: 0.3849 - accuracy: 0.9310
Epoch 37/50
8/8 [===

 >#3: 88.646 
Epoch 1/50
8/8 [==============================] - 0s 52ms/step - loss: 1.4574 - accuracy: 0.5137
Epoch 2/50
8/8 [==============================] - 0s 50ms/step - loss: 1.3980 - accuracy: 0.5027
Epoch 3/50
8/8 [==============================] - 0s 50ms/step - loss: 1.3377 - accuracy: 0.5520
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 1.3195 - accuracy: 0.4918
Epoch 5/50
8/8 [==============================] - 0s 49ms/step - loss: 1.2901 - accuracy: 0.5016
Epoch 6/50
8/8 [==============================] - 0s 50ms/step - loss: 1.2493 - accuracy: 0.5235
Epoch 7/50
8/8 [==============================] - 0s 49ms/step - loss: 1.2201 - accuracy: 0.5060
Epoch 8/50
8/8 [==============================] - 0s 50ms/step - loss: 1.2118 - accuracy: 0.4874
Epoch 9/50
8/8 [==============================] - 0s 49ms/step - loss: 1.1800 - accuracy: 0.4852
Epoch 10/50
8/8 [==============================] - 0s 49ms/step - loss: 1.1643 - accuracy: 0.4929
Epoch 11/50
8/8

8/8 [==============================] - 0s 50ms/step - loss: 0.9165 - accuracy: 0.5422
Epoch 27/50
8/8 [==============================] - 0s 50ms/step - loss: 0.9049 - accuracy: 0.5531
Epoch 28/50
8/8 [==============================] - 0s 49ms/step - loss: 0.8864 - accuracy: 0.5685
Epoch 29/50
8/8 [==============================] - 0s 49ms/step - loss: 0.8491 - accuracy: 0.6024
Epoch 30/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7868 - accuracy: 0.6813
Epoch 31/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7376 - accuracy: 0.7251
Epoch 32/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7015 - accuracy: 0.7590
Epoch 33/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6491 - accuracy: 0.7985
Epoch 34/50
8/8 [==============================] - 0s 51ms/step - loss: 0.6069 - accuracy: 0.8204
Epoch 35/50
8/8 [==============================] - 0s 51ms/step - loss: 0.5797 - accuracy: 0.8302
Epoch 36/50
8/8 [===============